# Module 4 Homework

## More Walmart Cleaning

The file **Walmart_United_States\_&\_Canada.csv** contains data on all
Walmarts, Sam's Clubs and Murphy USA gas/diesel in the USA and Canada.
Note that Gas/Diesel and No Over Night Parking (NOP) are indicated if
known. These data can be obtained from the site
<http://www.poi-factory.com/node/25560>.


**Before you start.** You started cleaning this data set in Activity 4.1.  Start by copying over your code and fixing the encoding issue.

In [1]:
# Copy your code from Activity 4.1 here
import pandas as pd
from dfply import *

from more_dfply import case_when, ifelse
from more_dfply.facets import text_facet, text_filter



walmart_data1 = pd.read_csv("./data/Walmart_United_States_&_Canada.csv", names=['Latitude', 'Longitude', 'Store_Number', 'Address'], encoding="utf-8")
walmart_data1.head()

(walmart_data1
 >> select(X.Store_Number)
 >> filter_by(text_filter(X.Store_Number, 'Gas/Diesel'))
 >> filter_by(text_filter(X.Store_Number, 'Gas'))
 >> head()
)



,Store_Number
70,"Murphy: USA; #7687,Gas/Diesel,"
71,"Walmart Supercenter; #0423,Gas/Diesel,"
75,"Walmart Supercenter; #1091,Gas/Diesel,"
76,"Murphy: USA; #7394,Gas/Diesel,"
79,"Walmart Supercenter; #0306,Gas/Diesel,"


In [2]:
walmart_data1.head()

,Latitude,Longitude,Store_Number,Address
0,-114.005671,51.262567,"Walmart Supercentre; #1050,","2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-..."
1,-111.900542,50.577939,"Walmart Supercentre; #3658,","917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111"
2,-114.039133,51.107253,"Walmart Supercentre; #3013,","1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40..."
3,-114.138488,51.040871,"Walmart Supercentre; #3009,Gas,","1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ..."
4,-114.028603,50.930551,"Walmart; #1144,","1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,..."


In [3]:
(walmart_data1
 >> select(X['Store_Number'])
 >> filter_by(~text_filter(X['Store_Number'], "(Walmart (Supercent(er|re))?)|(Sam's Club)|(Walmart)", regex=True))
 >> filter_by(X['Store_Number'].notna())
 )


/Users/drewolson/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/more_dfply/facets.py:30: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return col.str.contains(pattern, case=case, regex=regex, na=na)


,Store_Number
70,"Murphy: USA; #7687,Gas/Diesel,"
72,"Wm Nbrhd Mkt; #4756,Gas,"
73,"Murphy: USA; #5684,Gas,"
76,"Murphy: USA; #7394,Gas/Diesel,"
78,"Murphy: USA; #5624,Gas,"
...,...
6648,"Wm Nbrhd Mkt; #3145,"
6651,"Wm Nbrhd Mkt; #3052,"
6709,"Wm Nbrhd Mkt; #5697,"
6714,"Wm Nbrhd Mkt; #5695,"


In [4]:
(walmart_data1
 >> select(X.Store_Number)
 >> filter_by(~text_filter(X.Store_Number, 'Gas/Diesel'))
 >> filter_by(~text_filter(X.Store_Number, 'Gas'))
 >> head() #Show the rest
)

,Store_Number
0,"Walmart Supercentre; #1050,"
1,"Walmart Supercentre; #3658,"
2,"Walmart Supercentre; #3013,"
4,"Walmart; #1144,"
5,"Walmart Supercentre; #1136,"


In [5]:
walmart_data2 = (walmart_data1
 >> select(X.Store_Number)
 >> mutate(fuel = case_when((text_filter(X.Store_Number, 'Gas/Diesel'), "Gas/Diesel"),
                            ((text_filter(X.Store_Number, 'Gas'), "Gas")),
                            (True, "None")
                    ) )
#>> filter_by(~text_filter(X.Store_Number, 'Gas'))
# >> head() #Show the rest
)

In [6]:
#help(walmart_data1['Store_Number'].str.extract)  #Extract, pattern, capture groups. 

walmart_data3 = (walmart_data1
# >> select(X.Store_Number)
 >> mutate(Fuel = case_when((text_filter(X.Store_Number, 'Gas/Diesel'), "Gas/Diesel"),
                            ((text_filter(X.Store_Number, 'Gas'), "Gas")),
                            (True, "None")
                    ),
                    Store_Number = X.Store_Number.replace('Gas(/Diesel)?,', '', regex = True),
                    Num = X['Store_Number'].str.extract("#([0-9]+)"))
 >> mutate(Store_Number = X.Store_Number.replace('#([0-9]+)', '', regex = True))
 >> mutate(Store_Number = X.Store_Number.replace('(; ,)|(;,)','', regex = True))

#WM --> Walmart 
#capitals 
#Make supercenters all the same or use a dictionary and map it.

 >> mutate(store = X.Store_Number.str.lower().replace(r'wm\s? (.*)', r'Walmart \1', regex = True )) #\ to import capture group
 #>> mutate(Num = case_when(text_filter(X['Store_Number'], "(#[0-9]+)", regex=True))   )
                    
#>> filter_by(~text_filter(X.Store_Number, 'Gas'))
# >> head() #Show the rest
)
walmart_data3.store.unique()
#walmart_data3.head()

array(['walmart supercentre', 'walmart', 'walmart supercenter',
       'murphy: usa', 'Walmart nbrhd mkt', "sam's club",
       'Walmart pharmacy/clinic', 'walmart pickup & fuel',
       'Walmart nbrhd mkt,,', 'walmart; supercenter,,', 'walmart pickup',
       'murphy: usa;  ,', 'walmart fuel center,,',
       'walmart supercenter,,'], dtype=object)

In [7]:
walmart_data3.head()

,Latitude,Longitude,Store_Number,Address,Fuel,Num,store
0,-114.005671,51.262567,Walmart Supercentre,"2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",None,1050,walmart supercentre
1,-111.900542,50.577939,Walmart Supercentre,"917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",None,3658,walmart supercentre
2,-114.039133,51.107253,Walmart Supercentre,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40...",None,3013,walmart supercentre
3,-114.138488,51.040871,Walmart Supercentre,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ...",Gas,3009,walmart supercentre
4,-114.028603,50.930551,Walmart,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",None,1144,walmart


1.  Some of the address columns contain `(NOP)` to indicated *No overnight parking*.  Extract this information into a new indicator column, then remove it from the address column.

In [8]:
# Your code here
walmart_data4 = (walmart_data3
#>> select(X.Address)
# extract NOP from the address
>> mutate(NOP = ifelse(text_filter(X.Address, 'NOP'), 1, 0))
)

walmart_data4

,Latitude,Longitude,Store_Number,Address,Fuel,Num,store,NOP
0,-114.005671,51.262567,Walmart Supercentre,"2881 Main St SW,Airdrie ,AB T4B 3G5,(403) 945-...",None,1050,walmart supercentre,0
1,-111.900542,50.577939,Walmart Supercentre,"917 3rd St W,Brooks ,AB T1R 1L5,(403) 793-2111",None,3658,walmart supercentre,0
2,-114.039133,51.107253,Walmart Supercentre,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,(40...",None,3013,walmart supercentre,1
3,-114.138488,51.040871,Walmart Supercentre,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,(403) ...",Gas,3009,walmart supercentre,1
4,-114.028603,50.930551,Walmart,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,...",None,1144,walmart,0
...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,Walmart Supercenter,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",Gas,4471,walmart supercenter,0
6812,-108.379227,43.042858,Walmart Supercenter,"1733 N Federal Blvd,Riverton,WY,82501 ,,(307) ...",None,1457,walmart supercenter,0
6813,-109.251020,41.579761,Walmart Supercenter,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",None,1461,walmart supercenter,0
6814,-106.940967,44.779474,Walmart Supercenter,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",None,1508,walmart supercenter,1


2.  The address column contains the phone number of most of the stores.  Extract this information into a new column.  There are some issue with different patterns, so use the divide-and-conquer approach is advised.

In [9]:
walmart_data5 = (walmart_data4
>> mutate(Phone_number = X['Address'].str.extract('(\(\d\d\d\) \d\d\d-\d\d\d\d)'))

>> mutate(Address = X.Address.replace('(\(\d\d\d\) \d\d\d-\d\d\d\d)','', regex=True))
)
walmart_data5

,Latitude,Longitude,Store_Number,Address,Fuel,Num,store,NOP,Phone_number
0,-114.005671,51.262567,Walmart Supercentre,"2881 Main St SW,Airdrie ,AB T4B 3G5,",None,1050,walmart supercentre,0,(403) 945-1295
1,-111.900542,50.577939,Walmart Supercentre,"917 3rd St W,Brooks ,AB T1R 1L5,",None,3658,walmart supercentre,0,(403) 793-2111
2,-114.039133,51.107253,Walmart Supercentre,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,",None,3013,walmart supercentre,1,(403) 730-0990
3,-114.138488,51.040871,Walmart Supercentre,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,",Gas,3009,walmart supercentre,1,(403) 242-2205
4,-114.028603,50.930551,Walmart,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,",None,1144,walmart,0,(403) 225-6638
...,...,...,...,...,...,...,...,...,...
6811,-107.209281,41.792084,Walmart Supercenter,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",Gas,4471,walmart supercenter,0,(307) 417-3001
6812,-108.379227,43.042858,Walmart Supercenter,"1733 N Federal Blvd,Riverton,WY,82501 ,,",None,1457,walmart supercenter,0,(307) 856-3261
6813,-109.251020,41.579761,Walmart Supercenter,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",None,1461,walmart supercenter,0,(307) 362-1957
6814,-106.940967,44.779474,Walmart Supercenter,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",None,1508,walmart supercenter,1,(307) 674-6492


3.  Extract the country from the address column.

In [10]:
# Your code here
walmart_data6 = (walmart_data5
>> select(X.Address)
>> mutate(Country = case_when((text_filter(X.Address, '[A-Z]{2},\d{4,5}', regex=True)
                            , "USA"),
                             (text_filter(X.Address, '[A-Z]\d[A-Z]\s?\d?[A-Z]?\d?', regex=True)
                             , "Canada")
                           ))
)
walmart_data6

,Address,Country
0,"2881 Main St SW,Airdrie ,AB T4B 3G5,",Canada
1,"917 3rd St W,Brooks ,AB T1R 1L5,",Canada
2,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,",Canada
3,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,",Canada
4,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,",Canada
...,...,...
6811,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",USA
6812,"1733 N Federal Blvd,Riverton,WY,82501 ,,",USA
6813,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",USA
6814,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",USA


4.  Extract the state or province from the address columns

In [20]:
walmart_data7 = (walmart_data6
>> select(X.Address)
>> mutate(State_Province = case_when((text_filter(X.Address, '[A-Z]{2},\d{4,5}', regex=True),
            X.Address.str.extract('([A-Z]{2}),\d{4,5}', expand=False)),
            #text_filter(X.Address, '[A-Z]{2},\d{4,5}', regex=True))           
            )))

walmart_data7

,Address,State_Province
0,"2881 Main St SW,Airdrie ,AB T4B 3G5,",NaN
1,"917 3rd St W,Brooks ,AB T1R 1L5,",NaN
2,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,",NaN
3,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,",NaN
4,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,",NaN
...,...,...
6811,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",WY
6812,"1733 N Federal Blvd,Riverton,WY,82501 ,,",WY
6813,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",WY
6814,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",WY


In [23]:
walmart_data7 = (walmart_data6
>> select(X.Address)
>> mutate(State_Province = case_when((text_filter(X.Address, '[A-Z]{2},{4,5}', regex=True)
            , X.Address.str.extract('([A-Z]{2}),{4,5}', expand=False)),
            (text_filter(X.Address, '[A-Z]{2},{0,1}', regex=True),
            X.Address.str.extract('([A-Z]{2}),{0,1}', expand=False))    
            )))


walmart_data7



,Address,State_Province
0,"2881 Main St SW,Airdrie ,AB T4B 3G5,",SW
1,"917 3rd St W,Brooks ,AB T1R 1L5,",AB
2,"1110 57th Ave NE,Calgary ,(NOP),AB T2E 9B7,",NE
3,"1212 37 St SW,Calgary ,(NOP),AB T3C 1S3,",SW
4,"1221 Canyon Meadows Dr SE,Calgary ,AB T2J 6G2,",SE
...,...,...
6811,"2390 E Cedar St; I-80 Exit 214,Rawlins,WY,8230...",WY
6812,"1733 N Federal Blvd,Riverton,WY,82501 ,,",WY
6813,"201 Gateway Blvd; I-80 Exit 102,Rock Springs,W...",WY
6814,"1695 Coffeen Ave; I-90 Exit 25,Sheridan,WY,828...",WY


In [13]:
# Your code here
walmart_data7 = (walmart_data6
>> select(X.Address)
>> mutate(State_Province = X.Address.str.split(',').str.get(2)))
walmart_data7.tail(30)

,Address,State_Province
6786,"2900 Pike St,Parkersburg,WV,26101 ,,",WV
6787,"201 Greasy Ridge Rd; I-77 Exit 9,Princeton,WV,...",WV
6788,"1001 Warrior Way: I-75 Exit 36,Quincy,WV,25015 ,,",WV
6789,"200 Academy Dr; I-77 Exit 138,Ripley,WV,25271 ,,",WV
6790,"2500 Mountaineer Blvd,South Charleston,WV,2530...",WV
6791,"2700 Mountaineer Blvd,South Charleston,WV,2530...",WV
6792,"97 Williams Dr,Spencer,WV,25276 ,,",WV
6793,"200 Wal St,Summersville,WV,26651 ,,",WV
6794,"369 Scotts Fork-Bonnie Rd: I-79 exit 67,Sutton...",WV
6795,"450 Stewart Lane; I-70 Exit 10,Triadelphia,WV,...",WV


5. Combine all of your transformations into one pipe, then re-factor your code to be more readable.

In [14]:
# One pipe for all transformations.

In [15]:
# Refactored coded here (definitions/lambdas then the pipe)


In [16]:
# Write the results to a file named walmart_locations_clean.csv.  Make sure to include this file in your submission on D2L